In [1]:
import tensorflow as tf
from keras.datasets import cifar10
from keras.preprocessing.image import ImageDataGenerator
from keras.layers import Dense, Dropout, Activation, Flatten
from keras.layers import Conv2D, MaxPooling2D
# more info on callbakcs: https://keras.io/callbacks/ model saver is cool too.
from keras.callbacks import TensorBoard

from tensorflow import keras
from keras.models import Sequential
from keras.layers import *
from keras.applications.vgg16 import VGG16
from keras.preprocessing import image
from keras.applications.vgg16 import preprocess_input

from keras.preprocessing.image import ImageDataGenerator
from keras.callbacks import ModelCheckpoint

import pickle
import time

#tf.compat.v1.disable_eager_execution()

train_datagen = ImageDataGenerator(
        rescale=1./255,
        shear_range=0.2,
        zoom_range=0.2,
        horizontal_flip=False,
        validation_split=0.3)

test_datagen = ImageDataGenerator(rescale=1./255)

train_generator = train_datagen.flow_from_directory(
    directory=r"/home/trojan/Downloads/animals10/raw-img",
    target_size=(224, 224),
    color_mode="rgb",
    batch_size=32,
    class_mode="binary",
    shuffle=True,
    seed=42,
    subset="training"
)

valid_generator = train_datagen.flow_from_directory(
    directory=r"/home/trojan//Downloads/animals10/raw-img",
    target_size=(224, 224),
    color_mode="rgb",
    batch_size=32,
    class_mode="binary",
    shuffle=True,
    seed=42,
    subset="validation"
)

'''test_generator = test_datagen.flow_from_directory(
    directory=r"/home/trojan/Downloads/kagglecatsanddogs_3367a (copy)/test data",
    target_size=(224, 224),
    color_mode="rgb",
    batch_size=1,
    class_mode="binary",
    shuffle=False,
    seed=42
)'''

NAME = "animal-10-CNN-64*2-{}".format(int(time.time()))


model = Sequential()

model.add(VGG16(include_top=False, weights='imagenet', input_shape = (224,224, 3)))

model.add(Conv2D(256, (3, 3), padding='same'))
model.add(Activation('relu'))
model.add(MaxPooling2D(pool_size=(2, 2)))

model.add(Conv2D(256, (3, 3), padding='same'))
model.add(Activation('relu'))
model.add(MaxPooling2D(pool_size=(2, 2)))

model.add(Flatten())  # this converts our 3D feature maps to 1D feature vectors

model.add(Dense(256))
model.add(Activation('relu'))

model.add(Dense(10))
model.add(Activation('softmax'))

tensorboard = TensorBoard(log_dir="logs/{}".format(NAME))

model.compile(loss='sparse_categorical_crossentropy',
              optimizer='sgd',
              metrics=['accuracy'],
              )


STEP_SIZE_TRAIN=train_generator.n//train_generator.batch_size
STEP_SIZE_VALID=valid_generator.n//valid_generator.batch_size

filepath="weights-improvement-{epoch:02d}-{accuracy:.2f}.hdf5"
checkpoint = ModelCheckpoint(filepath, monitor='accuracy', verbose=1, save_best_only=True, mode='max')
callbacks_list = checkpoint

model.fit(train_generator,
                    steps_per_epoch=STEP_SIZE_TRAIN,
                    epochs=20,
                    callbacks=[callbacks_list, tensorboard])
                    


/home/trojan/.local/lib/python3.6/site-packages/tensorflow/python/framework/dtypes.py:516: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_qint8 = np.dtype([("qint8", np.int8, 1)])
/home/trojan/.local/lib/python3.6/site-packages/tensorflow/python/framework/dtypes.py:517: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_quint8 = np.dtype([("quint8", np.uint8, 1)])
/home/trojan/.local/lib/python3.6/site-packages/tensorflow/python/framework/dtypes.py:518: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_qint16 = np.dtype([("qint16", np.int16, 1)])
/home/trojan/.local/lib/python3.6/site-packages/tensorflow/python/framework/dtypes.py:519: 

Found 18331 images belonging to 10 classes.
Found 7848 images belonging to 10 classes.
Epoch 1/20
 84/572 [===>..........................] - ETA: 5:40 - loss: 1.8149 - accuracy: 0.3884

KeyboardInterrupt: 

In [ ]:
from keras.models import load_model 
model.save("animal-10_model.h5")
model.save_weights("animal-10_weights.h5")

In [6]:
model.evaluate_generator(generator=valid_generator,
steps=STEP_SIZE_VALID)

[2.2981090545654297, 0.18571428954601288]

In [7]:
valid_generator.reset()
pred=model.predict_generator(valid_generator,
steps=STEP_SIZE_VALID,
verbose=1)

245/245 [==============================] - 62s 253ms/step


In [8]:
import numpy as np
predicted_class_indices=np.argmax(pred,axis=1)


In [10]:
import pandas as pd
# Get classes by np.round
cl = np.round(pred)
# Get filenames (set shuffle=false in generator is important)
filenames=valid_generator.filenames


# Data frame
results=pd.DataFrame({"file":filenames,"pr":pred[:,0], "class":cl[:,0]})

results.to_csv("results4.csv",index=False)



ValueError: arrays must all be same length

In [ ]:
idx_to_cls = {v: k for k, v in valid_generator.class_indices.items()}
preds_cls = np.vectorize(idx_to_cls.get)(cl[:,0])
filenames_to_cls = list(zip(valid_generator.filenames))

In [ ]:
# Data frame
#filenames_to_cls = list(range(1, len(test_generator) + 1))
#filenames_to_cls.sort(key=lambda x: str(x))

results=pd.DataFrame({ "file":filenames_to_cls,"pr":pred[:,0], "class": cl[:,0], "class_name": preds_cls})

results.to_csv("results8.csv",index=False)